<a href="https://colab.research.google.com/github/finesketch/data_science/blob/main/Modern_Python_Cookbook/Reading_JSON_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to do it ...

```json
{ 
  "teams": [ 
    { 
      "name": "Abu Dhabi Ocean Racing", 
      "position": [ 
        1, 
        3, 
        2, 
        2, 
        1, 
        2, 
        5, 
        3, 
        5 
      ] 
    }, 
    ... 
  ], 
  "legs": [ 
    "ALICANTE - CAPE TOWN", 
    "CAPE TOWN - ABU DHABI", 
    "ABU DHABI - SANYA", 
    "SANYA - AUCKLAND", 
    "AUCKLAND - ITAJA\u00cd", 
    "ITAJA\u00cd - NEWPORT", 
    "NEWPORT - LISBON", 
    "LISBON - LORIENT", 
    "LORIENT - GOTHENBURG" 
  ] 
} 
```

Also, one of the strings contains a Unicode escape sequence, **\u00cd**, instead of the actual Unicode character, **Í**. This is a common technique used to encode characters beyond the 128 ASCII characters.

In [4]:
import json

In [1]:
from pathlib import Path
source_path = Path("race_result.json")

In [2]:
source_path

PosixPath('race_result.json')

In [5]:
document = json.loads(source_path.read_text())

In [6]:
document['teams'][0]['name']

'Abu Dhabi Ocean Racing'

In [7]:
document['legs'][5]

'ITAJAÍ - NEWPORT'

## How it works

A JSON document is a data structure in JavaScript Object Notation. JavaScript programs can parse the document trivially. Other languages must do a little more work to translate the JSON to a native data structure.

**A JSON document contains three kind of structures:**

* Object ther map to Python dictionaries: *{"key":"value"}*
* Arrays that map to Python lists: *[item1, item2, item3, ...]*
* Primitive values: *string, number, true/false, and null.*

## Deserialize JSON to Object in Python

## Serializing a complex data structure

Deserialization is the process of decoding the data that is in JSON format into native data type. In Python, deserialization decodes JSON data into a dictionary (data type in Python).

> Reference: https://www.geeksforgeeks.org/deserialize-json-to-object-in-python/#:~:text=Deserialization%20is%20the%20process%20of,document%20to%20a%20Python%20object.

> Reference: https://www.geeksforgeeks.org/modules-available-for-serialization-and-deserialization-in-python/



## Object Serialization in Python

Goal is to convert the JSON into a custom Python object, and vice versa.

> To do this, you would have to loop through the object and get all of its attributes one by one. This is fine for objects with few attributes, but as this number increases, your work gets tiring and repetitive.

> Reference: https://medium.com/swlh/object-serialization-and-deserialization-in-python-5fad3c2970a4


In [ ]:
class Person:
  def __init__(self, name, age):
    self.name = name
    self.age = age

person = Person(name='Bill', age=5)

return_value = {
    'name': person.name
    'age': person.age
}

## Python **"json"** module 

Reference: https://docs.python.org/3/library/json.html

# Serializing a Complex Data Structure

We can also create JSON documents from Python data structures. Because Python is extremely sophisticated and flexible, we can easily create Python data structures that cannot possibly be represented in JSON.

The serialization to JSON works out the best if we create Python objects that are limited to simple dict, list, str, int, float, bool, and None values.

In [9]:
import random
random.seed(1)
from collections import Counter

colors = (['red'] * 18) + (["black"] * 18) + (['green'] * 2)
data = Counter(random.choice(colors) for _ in range(100))
print(json.dumps(data, sort_keys=True, indent=2))

{
  "black": 53,
  "green": 7,
  "red": 40
}


In [10]:
colors

['red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'green',
 'green']

In [11]:
data

Counter({'black': 53, 'green': 7, 'red': 40})

In [13]:
output_path = Path('some_path.json')
output_path.write_text(json.dumps(data, sort_keys=True, indent=2))

44

In [14]:
# datetime.datetime object does not serialize easily

import datetime

example_date = datetime.datetime(2014,6,7,8,9,10)
document = {'date': example_date}
document

{'date': datetime.datetime(2014, 6, 7, 8, 9, 10)}

In [15]:
json.dumps(document)

TypeError: ignored

This shows that objects that cannot be serialized will raise a TypeError exception. Avoiding this exception can done in one of two ways. We can either convert the data before building the document, or we can add a hook to the JSON serialization process.

This uses the ISO format for dates to create a string that can be serialized. An application that reads this data can then convert the string back into a datetime object.

In [16]:
document_converted = {'date': example_date.isoformat()}
json.dumps(document_converted)

'{"date": "2014-06-07T08:09:10"}'

The other technique for serializing complex data is to provide a default function that's used automatically during serialization.

In [17]:
def default_date(object):
  if isinstance(object, datetime.datetime):
    return example_date.isoformat()
  return object

document = {'date': example_date}
print(json.dumps(document, default=default_date, indent=2))

{
  "date": "2014-06-07T08:09:10"
}


In any given application, we'll need to expand this function to handle any of the more complex Python objects that we might want to serialize in JSON notation. If there are a large number of very complex data structures, we often want a somewhat more general solution than meticulously converting each object to something serializable. There are a number of design patterns for including type information along with serialized details of an object's state.

# Deserializing a Complex Data Structure

When deserializing JSON to create Python objects, there's another hook that can be used to convert data from a JSON dictionary into a more complex Python object. This is called the **object_hook** and it is used during json.loads() processing to examine each complex object to see if something else should be created from that dict.


In [20]:
def as_date(object):
  if 'date' in object:
    return datetime.datetime.strptime(object['date'], '%Y-%m-%dT%H:%M:%S')
  return object

In [21]:
source = '''{"date": "2014-06-07T08:09:10"}'''
json.loads(source, object_hook=as_date)

datetime.datetime(2014, 6, 7, 8, 9, 10)

In a larger context, this particular example of handling dates isn't ideal. The presence of a single 'date' field to indicate a date object could lead to problems with more complex objects being de-serialized using this as_date() function.

We may also want to design our application classes to provide additional methods to help with serialization. A class might include a to_json() method that will serialize the objects in a uniform way. This method might provide class information. It can avoid serializing any derived attributes or computed properties. Similarly, we might need to provide a static from_json() method that can be used to determine if a given dictionary object is actually an instance of the given class.